# 🎬 Agente Recomendador de Películas y Series

Sistema de recomendaciones basado en Agentic RAG con ~20k películas y series.

**Stack:**
- LLM: Llama 3.1 8B (HuggingFace)
- Vector Store: Pinecone
- Framework: LangChain + LangGraph
- Interfaz: Gradio

## 1. Setup & Instalación

In [1]:
!pip uninstall -y pinecone-client pinecone
!pip install -q langchain==0.3.7 langchain-core==0.3.18 langgraph==0.2.19
!pip install -q langchain-community langchain-pinecone langchain-huggingface
!pip install -q sentence-transformers "pinecone-client>=7.0.0,<8.0.0"
!pip install -q huggingface-hub transformers torch
!pip install -q gradio pandas numpy tqdm

Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.2.13 requires pinecone[asyncio]<8.0.0,>=6.0.0, which is not installed.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.18 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.2 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.18 which is incompatible.
langchain-huggingface 1.2.0 requires langchain-core<2.0.0,>=1.2.0, but you have langchain-core 0.3.18 which is incompatible.
langchain-openai 1.1.4 requires langchain-core<2.0.0,>=1.2.1, but you have langchain-core 0.3.18 which is incompatible.
langchain-pinecone 0.2.13 requires langchain-core<2.0.0,>=0.3.34, but you have langchain-core 0.3.18 whi

## 2. Imports y Configuración

In [2]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime
from typing import Optional, List
import warnings
warnings.filterwarnings('ignore')

print("Imports básicos cargados")

Imports básicos cargados


## 3. Carga de Datasets

Cargaremos 3 datasets:
- IMDB Top 1000: 1,000 películas clásicas
- IMDB Films/TV: 11,414 registros mixtos
- Netflix: 7,789 películas + series

In [3]:
# Cargar datasets
df_imdb_top = pd.read_csv('Datasets/imdb_top_1000.csv')
df_imdb_films = pd.read_csv('Datasets/imdb Tv Series and Films.csv')
df_netflix = pd.read_csv('Datasets/NetFlix.csv')

print(f"IMDB Top 1000: {len(df_imdb_top):,} registros")
print(f"IMDB Films/TV: {len(df_imdb_films):,} registros")
print(f"Netflix: {len(df_netflix):,} registros")
print(f"\n Total antes de unificar: {len(df_imdb_top) + len(df_imdb_films) + len(df_netflix):,} registros")

IMDB Top 1000: 1,000 registros
IMDB Films/TV: 11,414 registros
Netflix: 7,787 registros

 Total antes de unificar: 20,201 registros


## 4. Funciones de Limpieza de Datos

Estas funciones normalizarán y limpiarán los datos de los 3 datasets.

In [4]:
def detect_type_from_year(year_str):
    """
    Detecta si es Movie o TV Show basándose en el formato del año.
    TV shows: "2018–2023" o "2023–" (en curso)
    Movies: "2023"
    """
    if pd.isna(year_str):
        return "Movie"  # Default
    
    year_str = str(year_str).strip()
    
    # TV shows tienen rangos con guiones
    if '–' in year_str or '—' in year_str or '-' in year_str:
        return "TV Show"
    
    return "Movie"


def extract_year_start(year_str):
    """
    Extrae el año de inicio de '2018–2023' o '2023'
    """
    if pd.isna(year_str):
        return 0
    
    year_str = str(year_str).strip()
    
    # Normalizar guiones
    year_str = year_str.replace('–', '-').replace('—', '-')
    
    # Quitar espacios y guiones finales
    year_str = year_str.replace('- ', f'-{datetime.now().year}')
    
    # Split y tomar primer año
    parts = year_str.split('-')
    
    try:
        return int(parts[0])
    except:
        return 0


def extract_year_end(year_str):
    """
    Extrae el año final, o año de inicio si es single year.
    Para series en curso ("2023–"), usa año actual.
    """
    if pd.isna(year_str):
        return 0
    
    year_str = str(year_str).strip()
    
    # Normalizar guiones
    year_str = year_str.replace('–', '-').replace('—', '-')
    
    year_str = year_str.replace('- ', f'-{datetime.now().year}')
    year_str = year_str.replace('-\s*$', f'-{datetime.now().year}')
    
    parts = year_str.split('-')
    
    try:
        if len(parts) > 1:
            return int(parts[1]) if parts[1].strip() else datetime.now().year
        return int(parts[0])
    except:
        return 0


def clean_cast_field(cast_str, field_type):
    """
    Extrae Director o Stars del formato messy de IMDB Films:
    'Directors:, Name, | , Stars:, Name1, , Name2, ,'
    
    Args:
        cast_str: String con formato IMDB
        field_type: 'Director' o 'Stars'
    """
    if pd.isna(cast_str):
        return "N/A"
    
    cast_str = str(cast_str)
    
    if field_type == 'Director':
        # Buscar entre "Director(s):" y "|"
        if 'Director' in cast_str:
            match = re.search(r'Directors?:,?\s*([^,|]+)', cast_str)
            if match:
                director = match.group(1).strip()
                # Limpiar residuos
                director = director.replace('|', '').strip()
                return director if director else "N/A"
        return "N/A"
    
    elif field_type == 'Stars':
        # Extraer después de "Stars:"
        if 'Stars:' in cast_str:
            stars_part = cast_str.split('Stars:')[1]
            # Limpiar comas extras y espacios
            stars = [s.strip() for s in stars_part.split(',') if s.strip() and s.strip() != '|']
            # Tomar top 4
            return ', '.join(stars[:4]) if stars else "N/A"
        return "N/A"
    
    return "N/A"


def parse_duration(duration_str):
    """
    Parse '140 min' → 140
    """
    if pd.isna(duration_str):
        return 0
    
    duration_str = str(duration_str).strip()
    match = re.search(r'(\d+)', duration_str)
    
    return int(match.group(1)) if match else 0


def parse_netflix_duration(duration_str):
    """
    Parse Netflix duration:
    - '4 Seasons' → 240 (4 * 60)
    - '143 min' → 143
    """
    if pd.isna(duration_str):
        return 0
    
    duration_str = str(duration_str).strip()
    
    if 'Season' in duration_str:
        # Estimar: 1 season ≈ 60 min promedio
        match = re.search(r'(\d+)', duration_str)
        seasons = int(match.group(1)) if match else 1
        return seasons * 60
    else:
        match = re.search(r'(\d+)', duration_str)
        return int(match.group(1)) if match else 0


def convert_rating_to_numeric(rating_str):
    """
    Convierte ratings a escala numérica 0-10:
    - Ratings numéricos (IMDB): pasar directo
    - TV-MA, R: ~7.0
    - PG-13: ~6.5
    - PG, TV-PG: ~6.0
    - G, TV-G: ~5.5
    """
    if pd.isna(rating_str):
        return 0.0
    
    rating_str = str(rating_str).strip()
    
    # Intentar numérico primero
    try:
        return float(rating_str)
    except:
        pass
    
    # Mapeo de certificados → ratings estimados
    rating_map = {
        'TV-MA': 7.0, 'R': 7.0, 'NC-17': 7.5,
        'TV-14': 6.5, 'PG-13': 6.5,
        'TV-PG': 6.0, 'PG': 6.0,
        'TV-G': 5.5, 'G': 5.5, 'TV-Y': 5.5,
        'NR': 0.0, 'UR': 0.0, 'N/A': 0.0,
        'A': 7.0, 'UA': 6.5, 'U': 6.0,  # Indian ratings
        'Passed': 6.0, 'Approved': 6.0
    }
    
    return rating_map.get(rating_str, 0.0)


def normalize_genres(genres_str):
    """
    Normaliza géneros entre datasets:
    - Remueve prefijos "TV Shows", "Movies", "International"
    - Estandariza: "Sci-Fi" → "Science Fiction"
    - Retorna top 3 géneros
    """
    if pd.isna(genres_str):
        return "Unknown"
    
    # Remover prefijos de Netflix
    genres_str = str(genres_str)
    genres_str = genres_str.replace('Movies', '').replace('TV Shows', '')
    genres_str = genres_str.replace('International', '').replace('British', '')
    
    genres = [g.strip() for g in genres_str.split(',') if g.strip()]
    
    # Estandarizar variaciones
    genre_map = {
        'Sci-Fi': 'Science Fiction',
        'Romantic': 'Romance',
        'Comedies': 'Comedy',
        'Thrillers': 'Thriller',
        'Dramas': 'Drama',
        'Documentaries': 'Documentary',
        'Kids': 'Family',
        "Children's": 'Family'
    }
    
    normalized = []
    for g in genres[:3]: 
        g_clean = g.replace('TV', '').strip()
        normalized.append(genre_map.get(g_clean, g_clean))
    
    return ', '.join(normalized) if normalized else "Unknown"


print("Funciones de limpieza definidas")

Funciones de limpieza definidas


## 5. Unificación de IMDB Top 1000

In [5]:
unified_imdb_top = []

print("Procesando IMDB Top 1000...")

for idx, row in df_imdb_top.iterrows():
      try:
          try:
              year_val = int(row['Released_Year'])
          except:
              year_val = 0

          votes_val = 0
          if pd.notna(row['No_of_Votes']):
              votes_str = str(row['No_of_Votes']).replace(',', '')
              try:
                  votes_val = int(votes_str)
              except:
                  votes_val = 0

          # Manejar rating
          try:
              rating_val = float(row['IMDB_Rating'])
          except:
              rating_val = 0.0

          record = {
              'content_id': f"imdb_top_{idx:04d}",
              'source': 'imdb_top_1000',
              'title': row['Series_Title'],
              'type': 'Movie',
              'year': str(year_val) if year_val > 0 else 'N/A',
              'year_start': year_val,
              'year_end': year_val,
              'genres': normalize_genres(row['Genre']),
              'rating': str(row['IMDB_Rating']),
              'rating_numeric': rating_val,
              'duration': row['Runtime'],
              'duration_minutes': parse_duration(row['Runtime']),
              'description': row['Overview'] if pd.notna(row['Overview']) else 'N/A',
              'director': row['Director'] if pd.notna(row['Director']) else 'N/A',
              'cast': f"{row['Star1']}, {row['Star2']}, {row['Star3']}, {row['Star4']}",
              'country': 'N/A',
              'certificate': row['Certificate'] if pd.notna(row['Certificate']) else 'N/A',
              'votes': votes_val
          }
          unified_imdb_top.append(record)
      except Exception as e:
          print(f"Error en fila {idx}: {e}")
          continue

df_unified_imdb_top = pd.DataFrame(unified_imdb_top)
print(f"IMDB Top 1000 unificado: {len(df_unified_imdb_top):,} registros")


Procesando IMDB Top 1000...
IMDB Top 1000 unificado: 1,000 registros


## 6. Unificación de IMDB Films/TV

In [6]:
unified_imdb_films = []

print("Procesando IMDB Films/TV...")

for idx, row in df_imdb_films.iterrows():
      if (idx + 1) % 1000 == 0:
          print(f"   Procesados {idx + 1:,}/{len(df_imdb_films):,}...")

      try:
          record = {
              'content_id': row['IMDb ID'] if pd.notna(row['IMDb ID']) else f"imdb_films_{idx:05d}",
              'source': 'imdb_films',
              'title': row['Title'],
              'type': detect_type_from_year(row['Release Year']),
              'year': str(row['Release Year']),
              'year_start': extract_year_start(row['Release Year']),
              'year_end': extract_year_end(row['Release Year']),
              'genres': normalize_genres(row['Genre']),
              'rating': str(row['Rating']) if pd.notna(row['Rating']) else 'N/A',
              'rating_numeric': float(row['Rating']) if pd.notna(row['Rating']) else 0.0,
              'duration': row['Runtime'] if pd.notna(row['Runtime']) else 'N/A',
              'duration_minutes': parse_duration(row['Runtime']),
              'description': row['Synopsis'] if pd.notna(row['Synopsis']) else 'N/A',
              'director': clean_cast_field(row['Cast'], 'Director'),
              'cast': clean_cast_field(row['Cast'], 'Stars'),
              'country': 'N/A',
              'certificate': row['Certificate'] if pd.notna(row['Certificate']) else 'N/A',
              'votes': int(row['Number of Votes']) if pd.notna(row['Number of Votes']) else 0
          }
          unified_imdb_films.append(record)
      except Exception as e:
          print(f"Error en fila {idx}: {e}")
          continue

df_unified_imdb_films = pd.DataFrame(unified_imdb_films)
print(f"IMDB Films/TV unificado: {len(df_unified_imdb_films):,} registros")


Procesando IMDB Films/TV...
   Procesados 1,000/11,414...
   Procesados 2,000/11,414...
   Procesados 3,000/11,414...
   Procesados 4,000/11,414...
   Procesados 5,000/11,414...
   Procesados 6,000/11,414...
   Procesados 7,000/11,414...
   Procesados 8,000/11,414...
   Procesados 9,000/11,414...
   Procesados 10,000/11,414...
   Procesados 11,000/11,414...
IMDB Films/TV unificado: 11,414 registros


## 7. Unificación de Netflix

In [7]:
unified_netflix = []

print("Procesando Netflix...")

for idx, row in df_netflix.iterrows():
      if (idx + 1) % 1000 == 0:
          print(f"   Procesados {idx + 1:,}/{len(df_netflix):,}...")

      try:
          record = {
              'content_id': row['show_id'],
              'source': 'netflix',
              'title': row['title'],
              'type': row['type'],
              'year': str(row['release_year']),
              'year_start': int(row['release_year']),
              'year_end': int(row['release_year']),
              'genres': normalize_genres(row['genres']),
              'rating': row['rating'] if pd.notna(row['rating']) else 'N/A',
              'rating_numeric': convert_rating_to_numeric(row['rating']),
              'duration': row['duration'] if pd.notna(row['duration']) else 'N/A',
              'duration_minutes': parse_netflix_duration(row['duration']),
              'description': row['description'] if pd.notna(row['description']) else 'N/A',
              'director': row['director'] if pd.notna(row['director']) else 'N/A',
              'cast': row['cast'] if pd.notna(row['cast']) else 'N/A',
              'country': row['country'] if pd.notna(row['country']) else 'N/A',
              'certificate': row['rating'] if pd.notna(row['rating']) else 'N/A',
              'votes': 0
          }
          unified_netflix.append(record)
      except Exception as e:
          print(f"Error en fila {idx}: {e}")
          continue

df_unified_netflix = pd.DataFrame(unified_netflix)
print(f"Netflix unificado: {len(df_unified_netflix):,} registros")


Procesando Netflix...
   Procesados 1,000/7,787...
   Procesados 2,000/7,787...
   Procesados 3,000/7,787...
   Procesados 4,000/7,787...
   Procesados 5,000/7,787...
   Procesados 6,000/7,787...
   Procesados 7,000/7,787...
Netflix unificado: 7,787 registros


## 8. Merge y Deduplicación

In [8]:
# Concatenar todos los DataFrames
df_all = pd.concat([df_unified_imdb_top, df_unified_imdb_films, df_unified_netflix], ignore_index=True)

print(f" Total antes de deduplicación: {len(df_all):,} registros")
print(f"   - IMDB Top: {len(df_unified_imdb_top):,}")
print(f"   - IMDB Films: {len(df_unified_imdb_films):,}")
print(f"   - Netflix: {len(df_unified_netflix):,}")

 Total antes de deduplicación: 20,201 registros
   - IMDB Top: 1,000
   - IMDB Films: 11,414
   - Netflix: 7,787


In [9]:
def deduplicate_records(unified_df):
    """
    Remueve duplicados priorizando:
    1. IMDB IDs (más autoritativo)
    2. Mayor cantidad de votes
    3. Metadata más completa
    """
    print("\n Deduplicando registros...")
    
    # Crear clave de deduplicación
    unified_df['dedup_key'] = (
        unified_df['title'].str.lower().str.strip() + '_' + 
        unified_df['year_start'].astype(str) + '_' + 
        unified_df['type']
    )
    
    # Función de scoring (mayor = mejor)
    def score_record(row):
        score = 0
        
        # IMDB ID = más confiable
        if row['content_id'].startswith('tt'):
            score += 100
        
        # Votes = popularidad
        score += min(row['votes'] / 1000, 50)
        
        # Descripción completa
        if row['description'] != 'N/A' and len(row['description']) > 50:
            score += 20
        
        # Director disponible
        if row['director'] != 'N/A':
            score += 10
        
        # Cast disponible
        if row['cast'] != 'N/A':
            score += 10
        
        return score
    
    unified_df['quality_score'] = unified_df.apply(score_record, axis=1)
    
    # Ordenar por score y quedarse con el mejor de cada grupo
    deduplicated = unified_df.sort_values('quality_score', ascending=False).drop_duplicates(
        subset=['dedup_key'], keep='first'
    )
    
    duplicates_removed = len(unified_df) - len(deduplicated)
    print(f"    Duplicados removidos: {duplicates_removed:,}")
    print(f"    Registros finales: {len(deduplicated):,}")
    
    # Limpiar columnas temporales
    deduplicated = deduplicated.drop(columns=['dedup_key', 'quality_score'])
    
    return deduplicated


# Aplicar deduplicación
df_unified = deduplicate_records(df_all)


 Deduplicando registros...
    Duplicados removidos: 2,857
    Registros finales: 17,344


## 9. Validación de Calidad

In [10]:
print("\n VALIDACIÓN DE CALIDAD DEL DATASET UNIFICADO\n" + "="*60)

print(f"\n Registros totales: {len(df_unified):,}")
print(f"\n Por tipo:")
print(f"   - Películas: {(df_unified['type'] == 'Movie').sum():,}")
print(f"   - Series: {(df_unified['type'] == 'TV Show').sum():,}")

print(f"\n Calidad de descripciones:")
print(f"   - Con descripción: {(df_unified['description'] != 'N/A').sum():,}")
print(f"   - Sin descripción: {(df_unified['description'] == 'N/A').sum():,}")

print(f"\n Ratings:")
print(f"   - Rating promedio: {df_unified['rating_numeric'].mean():.2f}/10")
print(f"   - Con rating > 0: {(df_unified['rating_numeric'] > 0).sum():,}")

print(f"\n Rango de años:")
print(f"   - Año más antiguo: {df_unified['year_start'].min()}")
print(f"   - Año más reciente: {df_unified['year_end'].max()}")

print(f"\n Top 10 géneros:")
for genre, count in df_unified['genres'].value_counts().head(10).items():
    print(f"   - {genre}: {count:,}")

print(f"\n Por fuente:")
for source, count in df_unified['source'].value_counts().items():
    print(f"   - {source}: {count:,}")

print("\n" + "="*60)


 VALIDACIÓN DE CALIDAD DEL DATASET UNIFICADO

 Registros totales: 17,344

 Por tipo:
   - Películas: 13,345
   - Series: 3,999

 Calidad de descripciones:
   - Con descripción: 17,344
   - Sin descripción: 0

 Ratings:
   - Rating promedio: 6.34/10
   - Con rating > 0: 16,994

 Rango de años:
   - Año más antiguo: 0
   - Año más reciente: 2025

 Top 10 géneros:
   - Action, Crime, Drama: 1,017
   - Animation, Action, Adventure: 899
   - Drama: 649
   - Action, Adventure, Drama: 593
   - Action, Adventure, Comedy: 555
   - Documentary: 506
   - Action, Comedy, Crime: 390
   - Comedy: 385
   - Action, Crime, Thriller: 371
   - Action, Adventure, Fantasy: 364

 Por fuente:
   - imdb_films: 9,300
   - netflix: 7,191
   - imdb_top_1000: 853



## 10. Guardar Dataset Unificado

In [11]:
# Guardar a CSV
output_file = 'unified_dataset.csv'
df_unified.to_csv(output_file, index=False)

print(f"Dataset unificado guardado en: {output_file}")
print(f"Tamaño del archivo: {os.path.getsize(output_file) / 1024 / 1024:.2f} MB")
print(f" {len(df_unified):,} registros × {len(df_unified.columns)} columnas")

Dataset unificado guardado en: unified_dataset.csv
Tamaño del archivo: 6.02 MB
 17,344 registros × 18 columnas


## 11. Preview de Registros

In [12]:
# Mostrar algunos ejemplos
print("\n Ejemplo de película:")
movie_sample = df_unified[df_unified['type'] == 'Movie'].iloc[0]
for col in ['title', 'year', 'genres', 'rating_numeric', 'description']:
    print(f"   {col}: {movie_sample[col]}")

print("\n Ejemplo de serie:")
tv_sample = df_unified[df_unified['type'] == 'TV Show'].iloc[0]
for col in ['title', 'year', 'genres', 'rating_numeric', 'description']:
    print(f"   {col}: {tv_sample[col]}")

df_unified.head(3)


 Ejemplo de película:
   title: Spider-Man: Into the Spider-Verse
   year: 2018
   genres: Animation, Action, Adventure
   rating_numeric: 8.4
   description: Teen Miles Morales becomes the Spider-Man of his universe and must join with five spider-powered individuals from other dimensions to stop a threat for all realities.

 Ejemplo de serie:
   title: Barry
   year: 2018–2023
   genres: Action, Comedy, Crime
   rating_numeric: 8.4
   description: A hit man from the Midwest moves to Los Angeles and gets caught up in the city's theatre arts scene.


,content_id,source,title,type,year,year_start,year_end,genres,rating,rating_numeric,duration,duration_minutes,description,director,cast,country,certificate,votes
11413,tt4633694,imdb_films,Spider-Man: Into the Spider-Verse,Movie,2018,2018,2018,"Animation, Action, Adventure",8.4,8.4,117 min,117,Teen Miles Morales becomes the Spider-Man of h...,Bob Persichetti,"Shameik Moore, Jake Johnson, Hailee Steinfeld,...",N/A,PG,575321
1495,tt1408101,imdb_films,Star Trek Into Darkness,Movie,2013,2013,2013,"Action, Adventure, Science Fiction",7.7,7.7,132 min,132,After the crew of the Enterprise find an unsto...,J.J. Abrams,"Chris Pine, Zachary Quinto, Zoe Saldana, Bened...",N/A,PG-13,489502
2538,tt0401855,imdb_films,Underworld: Evolution,Movie,2006,2006,2006,"Action, Fantasy, Thriller",6.7,6.7,106 min,106,"Picking up directly from the previous movie, v...",Len Wiseman,"Kate Beckinsale, Scott Speedman, Bill Nighy, T...",N/A,R,204681


---

## FASE 1 COMPLETADA

**Siguiente paso:** Esperar aprobación de Llama 3.1 y continuar con:
- Fase 2: Vector Store (Pinecone + Embeddings)
- Fase 3: Tools del agente
- Fase 4: StateGraph con LLM
- Fase 5: Interfaz Gradio

---

# FASE 2: Vector Store (Pinecone + Embeddings)

Configuraremos Pinecone y subiremos los ~20k registros con embeddings para búsqueda semántica.

In [ ]:
!pip install -q langchain-huggingface
!pip install -q langchain-pinecone
!pip install -q tqdm
!pip install python-dotenv

print("Dependencias de Fase 2 instaladas")



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\tatic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\tatic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Dependencias de Fase 2 instaladas



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\tatic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 12. Imports para RAG y Vector Store

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
import pinecone
from tqdm import tqdm
import time
import os

print("Imports de LangChain y Pinecone cargados")

Imports de LangChain y Pinecone cargados


## 13. Configuración de API Keys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

 # API Keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HUGGINGFACE_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")

  # Configuración del índice Pinecone
PINECONE_INDEX_NAME = "movie-recommender"
PINECONE_DIMENSION = 384
NAMESPACE = "movies-series"

print("Congiguración Cargada")

Índice 'movie-recommender' ya existe
Pinecone configurado correctamente


## 14. Inicializar Embeddings Model

In [16]:
 # Inicializar modelo de embeddings
embeddings = HuggingFaceEmbeddings(
model_name="sentence-transformers/all-MiniLM-L6-v2",
model_kwargs={'device': 'cpu'},
encode_kwargs={'normalize_embeddings': True}
  )

  # Test de embeddings
test_text = "A thrilling science fiction movie"
test_embedding = embeddings.embed_query(test_text)

print(f"Modelo de embeddings cargado")
print(f"Dimension: {len(test_embedding)}")
print(f"Primeros 5 valores: {test_embedding[:5]}")


Modelo de embeddings cargado
Dimension: 384
Primeros 5 valores: [-0.05855903774499893, 0.021344512701034546, -0.03782050684094429, 0.055703677237033844, -0.04008974879980087]


## 15. Conectar a Pinecone y Crear Índice


In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

  # Verificar si el índice existe, si no, crearlo
existing_indexes = [index.name for index in pc.list_indexes()]

if PINECONE_INDEX_NAME not in existing_indexes:
      print(f"Creando índice '{PINECONE_INDEX_NAME}'...")
      pc.create_index(
          name=PINECONE_INDEX_NAME,
          dimension=PINECONE_DIMENSION,
          metric=PINECONE_METRIC,
          spec=ServerlessSpec(
              cloud='aws',
              region='us-east-1'
          )
      )
      print(f"Índice creado. Esperando 30 segundos para que se inicialice...")
      time.sleep(30)
else:
      print(f"Índice '{PINECONE_INDEX_NAME}' ya existe")

  # Obtener estadísticas del índice
index = pc.Index(PINECONE_INDEX_NAME)
stats = index.describe_index_stats()

print(f"\nEstadísticas del índice:")
print(f"   Nombre: {PINECONE_INDEX_NAME}")
print(f"   Dimensión: {PINECONE_DIMENSION}")
print(f"   Vectores actuales: {stats.total_vector_count}")


Índice 'movie-recommender' ya existe

Estadísticas del índice:
   Nombre: movie-recommender
   Dimensión: 384
   Vectores actuales: 138592


## 16. Cargar Dataset Unificado

In [18]:
 # Cargar dataset unificado (si aún no está en memoria)
if 'df_unified' not in locals():
      df_unified = pd.read_csv('unified_dataset.csv')
      print(f"Dataset cargado desde CSV: {len(df_unified):,} registros")
else:
      print(f"Dataset ya en memoria: {len(df_unified):,} registros")

print(f"\nColumnas disponibles: {list(df_unified.columns)}")

Dataset ya en memoria: 17,344 registros

Columnas disponibles: ['content_id', 'source', 'title', 'type', 'year', 'year_start', 'year_end', 'genres', 'rating', 'rating_numeric', 'duration', 'duration_minutes', 'description', 'director', 'cast', 'country', 'certificate', 'votes']


## 17. Crear Documentos LangChain con Metadata Rica

Convertiremos cada película/serie en un Document con:
- **page_content:** Descripción del contenido
- **metadata:** Campos filtrables (tipo, géneros, año, rating, director, cast)

In [19]:
def create_document_from_row(row):
      """
      Convierte una fila del DataFrame en un Document de LangChain.
      """
      # Crear texto enriquecido para embeddings
      title = row['title']
      genres = row['genres']
      description = row['description'] if row['description'] != 'N/A' else ''
      content_type = row['type']
      year = row['year']

      # Texto combinado (contexto para embeddings)
      page_content = f"{title} ({year})\n"
      page_content += f"Tipo: {content_type}\n"
      page_content += f"Géneros: {genres}\n"
      if description:
          page_content += f"Descripción: {description}"

      # Metadata filtrable
      metadata = {
          'content_id': str(row['content_id']),
          'title': title,
          'type': content_type,
          'genres': genres,
          'year_start': int(row['year_start']) if row['year_start'] > 0 else 0,
          'rating_numeric': float(row['rating_numeric']),
          'duration_minutes': int(row['duration_minutes']),
          'director': str(row['director']) if row['director'] != 'N/A' else '',
          'source': str(row['source'])
      }

      return Document(page_content=page_content, metadata=metadata)
print("Creando documentos LangChain...")
documents = []

for idx, row in tqdm(df_unified.iterrows(), total=len(df_unified), desc="Procesando"):
      try:
          doc = create_document_from_row(row)
          documents.append(doc)
      except Exception as e:
          print(f"Error en fila {idx}: {e}")
          continue

print(f"\n{len(documents):,} documentos creados")

if documents:
      print("\nEjemplo de documento:")
      print(f"Content: {documents[0].page_content[:150]}...")
      print(f"Metadata: {documents[0].metadata}")



Creando documentos LangChain...


Procesando: 100%|██████████| 17344/17344 [00:00<00:00, 18749.15it/s]


17,344 documentos creados

Ejemplo de documento:
Content: Spider-Man: Into the Spider-Verse (2018)
Tipo: Movie
Géneros: Animation, Action, Adventure
Descripción: Teen Miles Morales becomes the Spider-Man of h...
Metadata: {'content_id': 'tt4633694', 'title': 'Spider-Man: Into the Spider-Verse', 'type': 'Movie', 'genres': 'Animation, Action, Adventure', 'year_start': 2018, 'rating_numeric': 8.4, 'duration_minutes': 117, 'director': 'Bob Persichetti', 'source': 'imdb_films'}


## 18. Subir Documentos a Pinecone

**Importante:** Este proceso puede tomar 5-10 minutos para ~20k documentos.

Subiremos en batches para optimizar la carga.

In [20]:
NAMESPACE = "movies-series"

print(f"Iniciando carga a Pinecone...")
print(f"   Índice: {PINECONE_INDEX_NAME}")
print(f"   Namespace: {NAMESPACE}")
print(f"   Total documentos: {len(documents):,}")
print(f"\nEsto puede tomar 5-10 minutos...\n")

# Obtener el objeto Index del cliente ya autenticado
index = pc.Index(PINECONE_INDEX_NAME)

# Subir usando PineconeVectorStore con el index autenticado
start_time = time.time()

try:
    vectorstore = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=NAMESPACE
    )
    
    elapsed = time.time() - start_time
    print(f"\nCarga completada en {elapsed/60:.2f} minutos")
    print(f"Vectores subidos: {len(documents):,}")
    
except Exception as e:
    print(f"\nError durante la carga: {e}")
    print("\nPosibles soluciones:")
    print("   1. Verifica que PINECONE_API_KEY sea correcta")
    print("   2. Asegúrate que el índice existe en Pinecone")
    print("   3. Verifica que la dimensión sea 384")
    print("   4. Revisa los rate limits de tu plan de Pinecone")
    
    print("\nIntentando configurar variable de entorno...")
    os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
    
    try:
        vectorstore = PineconeVectorStore.from_documents(
            documents=documents,
            embedding=embeddings,
            index_name=PINECONE_INDEX_NAME,
            namespace=NAMESPACE
        )
        elapsed = time.time() - start_time
        print(f"\nCarga completada en {elapsed/60:.2f} minutos")
        print(f"Vectores subidos: {len(documents):,}")
    except Exception as e2:
        print(f"\nError persistente: {e2}")
    
print("Creando documentos LangChain...")
documents = []

for idx, row in tqdm(df_unified.iterrows(), total=len(df_unified), desc="Procesando"):
      try:
          doc = create_document_from_row(row)
          documents.append(doc)
      except Exception as e:
          print(f"Error en fila {idx}: {e}")
          continue

print(f"\n{len(documents):,} documentos creados")

if documents:
      print("\nEjemplo de documento:")
      print(f"Content: {documents[0].page_content[:150]}...")
      print(f"Metadata: {documents[0].metadata}")


Iniciando carga a Pinecone...
   Índice: movie-recommender
   Namespace: movies-series
   Total documentos: 17,344

Esto puede tomar 5-10 minutos...


Error durante la carga: Pinecone API key must be provided in either `pinecone_api_key` or `PINECONE_API_KEY` environment variable

Posibles soluciones:
   1. Verifica que PINECONE_API_KEY sea correcta
   2. Asegúrate que el índice existe en Pinecone
   3. Verifica que la dimensión sea 384
   4. Revisa los rate limits de tu plan de Pinecone

Intentando configurar variable de entorno...

Carga completada en 4.76 minutos
Vectores subidos: 17,344
Creando documentos LangChain...


Procesando: 100%|██████████| 17344/17344 [00:00<00:00, 17426.04it/s]


17,344 documentos creados

Ejemplo de documento:
Content: Spider-Man: Into the Spider-Verse (2018)
Tipo: Movie
Géneros: Animation, Action, Adventure
Descripción: Teen Miles Morales becomes the Spider-Man of h...
Metadata: {'content_id': 'tt4633694', 'title': 'Spider-Man: Into the Spider-Verse', 'type': 'Movie', 'genres': 'Animation, Action, Adventure', 'year_start': 2018, 'rating_numeric': 8.4, 'duration_minutes': 117, 'director': 'Bob Persichetti', 'source': 'imdb_films'}


## 19. Verificar Vector Store con Búsquedas de Prueba

Realizaremos algunas búsquedas de prueba para confirmar que todo funciona correctamente.

In [21]:
 # Crear el vectorstore con embeddings  
embeddings = HuggingFaceEmbeddings(
model_name="sentence-transformers/all-MiniLM-L6-v2",
model_kwargs={'device': 'cpu'},
encode_kwargs={'normalize_embeddings': True} )

index = pc.Index(PINECONE_INDEX_NAME)

  # Crear el vectorstore de LangChain
vectorstore = PineconeVectorStore(
      index=index,
      embedding=embeddings,
      text_key="text",
      namespace=NAMESPACE
  )

print("Vectorstore configurado y listo para usar")


Vectorstore configurado y listo para usar


In [22]:
print("=" * 60)
print("Test 2: Búsqueda con filtros de metadata")
print("=" * 60 + "\n")

query = "historia de amor romántica"
filter_dict = {
      "type": {"$eq": "Movie"},
      "rating_numeric": {"$gte": 7.0},
      "year_start": {"$gte": 2010}
  }

results = vectorstore.similarity_search(
      query,
      k=5,
      namespace=NAMESPACE,
      filter=filter_dict
  )

print(f"Query: '{query}'")
print(f"Filtros: Películas, rating ≥ 7.0, año ≥ 2010\n")

for i, doc in enumerate(results, 1):
      print(f"{i}. {doc.metadata['title']} ({doc.metadata['year_start']})")
      print(f"   Géneros: {doc.metadata['genres']}")
      print(f"   Rating: {doc.metadata['rating_numeric']}/10")
      print()


Test 2: Búsqueda con filtros de metadata

Query: 'historia de amor romántica'
Filtros: Películas, rating ≥ 7.0, año ≥ 2010

1. Sí, Mi Amor (2020.0)
   Géneros: Comedy, Romance
   Rating: 7.0/10

2. Sí, Mi Amor (2020.0)
   Géneros: Comedy, Romance
   Rating: 7.0/10

3. Sí, Mi Amor (2020.0)
   Géneros: Comedy, Romance
   Rating: 7.0/10

4. Sí, Mi Amor (2020.0)
   Géneros: Comedy, Romance
   Rating: 7.0/10

5. Sí, Mi Amor (2020.0)
   Géneros: Comedy, Romance
   Rating: 7.0/10



In [23]:
print("=" * 60)
print("Test 3: Estadísticas del índice Pinecone")
print("=" * 60 + "\n")

index_stats = pc.Index(PINECONE_INDEX_NAME).describe_index_stats()

print(f"Total de vectores: {index_stats.total_vector_count:,}")
print(f"\nNamespaces:")
for ns, stats in index_stats.namespaces.items():
      print(f"   - {ns}: {stats.vector_count:,} vectores")

print(f"\nVector Store funcionando correctamente!")


Test 3: Estadísticas del índice Pinecone

Total de vectores: 155,936

Namespaces:
   - movies-series: 155,936 vectores

Vector Store funcionando correctamente!


---

## FASE 2 COMPLETADA

**Resumen:**
- Embeddings model configurado (all-MiniLM-L6-v2, dim=384)
- ~20k documentos creados con metadata rica
- Vectores subidos a Pinecone
- Búsquedas semánticas verificadas
- Filtros de metadata funcionando

**Siguiente paso:**
- Fase 3: Herramientas especializadas del agente (search_by_mood, search_by_criteria, etc.)

## 20. Imports para tools

In [24]:
from langchain.tools import tool
from typing import Optional, List, Dict, Any
import json

print("Imports para tools cargados")


Imports para tools cargados


 ## 21. Tool 1: Búsqueda Semántica


In [25]:
@tool
def semantic_search(query: str, content_type: Optional[str] = None, limit: int = 5) -> str:
      """
      Busca películas o series usando búsqueda semántica basada en descripción, mood o tema.

      Args:
          query: Descripción de lo que el usuario busca (ej: "películas de acción emocionantes", "series románticas tristes")
          content_type: Opcional. Filtrar por tipo: "Movie" o "TV Show"
          limit: Número máximo de resultados (default: 5)

      Returns:
          JSON string con los resultados encontrados
      """
      try:
          filter_dict = None
          if content_type:
              filter_dict = {"type": {"$eq": content_type}}

          # Realizar búsqueda semántica
          results = vectorstore.similarity_search(
              query,
              k=limit,
              namespace=NAMESPACE,
              filter=filter_dict
          )

          formatted_results = []
          for doc in results:
              formatted_results.append({
                  "title": doc.metadata['title'],
                  "type": doc.metadata['type'],
                  "genres": doc.metadata['genres'],
                  "year": doc.metadata['year_start'],
                  "rating": doc.metadata['rating_numeric'],
                  "director": doc.metadata.get('director', 'N/A'),
                  "description": doc.page_content.split("Descripción: ")[-1] if "Descripción: " in doc.page_content else "N/A"
              })

          return json.dumps(formatted_results, ensure_ascii=False, indent=2)

      except Exception as e:
          return json.dumps({"error": str(e)})


print("Tool 'semantic_search' creada")


Tool 'semantic_search' creada


 ## 22. Tool 2: Filtro por Metadata

In [26]:
@tool
def filter_by_metadata(
      content_type: Optional[str] = None,
      genre: Optional[str] = None,
      min_year: Optional[int] = None,
      max_year: Optional[int] = None,
      min_rating: Optional[float] = None,
      min_duration: Optional[int] = None,
      max_duration: Optional[int] = None,
      limit: int = 10
  ) -> str:
      """
      Filtra películas o series por metadata específica.

      Args:
          content_type: "Movie" o "TV Show"
          genre: Género a buscar (ej: "Action", "Comedy", "Drama")
          min_year: Año mínimo de lanzamiento
          max_year: Año máximo de lanzamiento
          min_rating: Rating mínimo (0-10)
          min_duration: Duración mínima en minutos
          max_duration: Duración máxima en minutos
          limit: Número máximo de resultados

      Returns:
          JSON string con los resultados filtrados
      """
      try:
          filter_dict = {}

          if content_type:
              filter_dict["type"] = {"$eq": content_type}

          if min_rating is not None:
              filter_dict["rating_numeric"] = {"$gte": min_rating}

          if min_year is not None:
              filter_dict["year_start"] = {"$gte": min_year}

          if min_duration is not None:
              if "duration_minutes" not in filter_dict:
                  filter_dict["duration_minutes"] = {}
              filter_dict["duration_minutes"]["$gte"] = min_duration

          if max_duration is not None:
              if "duration_minutes" not in filter_dict:
                  filter_dict["duration_minutes"] = {}
              filter_dict["duration_minutes"]["$lte"] = max_duration

          # Query genérico para búsqueda con filtros
          query_text = f"{content_type or 'contenido'} {genre or ''}"

          # Realizar búsqueda con filtros
          results = vectorstore.similarity_search(
              query_text,
              k=limit,
              namespace=NAMESPACE,
              filter=filter_dict if filter_dict else None
          )

          # Si se especificó género, filtrar manualmente (Pinecone no soporta contains en metadata)
          if genre:
              results = [r for r in results if genre.lower() in r.metadata['genres'].lower()]

          formatted_results = []
          for doc in results[:limit]:
              formatted_results.append({
                  "title": doc.metadata['title'],
                  "type": doc.metadata['type'],
                  "genres": doc.metadata['genres'],
                  "year": doc.metadata['year_start'],
                  "rating": doc.metadata['rating_numeric'],
                  "duration_minutes": doc.metadata['duration_minutes'],
                  "director": doc.metadata.get('director', 'N/A')
              })

          return json.dumps(formatted_results, ensure_ascii=False, indent=2)

      except Exception as e:
          return json.dumps({"error": str(e)})


print("Tool 'filter_by_metadata' creada")


Tool 'filter_by_metadata' creada


 ## 23. Tool 3: Top Rated


In [27]:
@tool
def get_top_rated(
      content_type: Optional[str] = None,
      genre: Optional[str] = None,
      year: Optional[int] = None,
      limit: int = 10
  ) -> str:
      """
      Obtiene las películas o series mejor calificadas con suficientes votos.

      Args:
          content_type: "Movie" o "TV Show"
          genre: Género específico (opcional)
          year: Año específico (opcional)
          limit: Número de resultados

      Returns:
          JSON string con el top contenido
      """
      try:
          filter_dict = {
              "rating_numeric": {"$gte": 5.0}
          }

          if content_type:
              filter_dict["type"] = {"$eq": content_type}

          if year:
              filter_dict["year_start"] = {"$eq": year}

          if content_type == "Movie":
              query_text = "movie"
          elif content_type == "TV Show":
              query_text = "television"
          else:
              query_text = "entertainment"

          # Generar embedding
          query_embedding = embeddings.embed_query(query_text)

          index = pc.Index(PINECONE_INDEX_NAME)

          query_response = index.query(
              vector=query_embedding,
              filter=filter_dict,
              top_k=10000,
              include_metadata=True,
              namespace=NAMESPACE
          )

          # Extraer resultados
          results = []
          seen_titles = set()

          # Para películas: mínimo 50,000 votos
          # Para series: mínimo 10,000 votos
          min_votes = 50000 if content_type == "Movie" else 10000

          for match in query_response['matches']:
              metadata = match['metadata']
              title = metadata.get('title', '').lower().strip()

              # Saltar duplicados
              if title in seen_titles:
                  continue

              votes = metadata.get('votes', 0)
              if votes == 0: 
                  votes = metadata.get('num_votes', 0)

              if votes < min_votes:
                  continue

              seen_titles.add(title)

              if genre:
                  genres = metadata.get('genres', '').lower()
                  if genre.lower() not in genres:
                      continue

              results.append({
                  "title": metadata.get('title', 'N/A'),
                  "type": metadata.get('type', 'N/A'),
                  "genres": metadata.get('genres', 'N/A'),
                  "year": metadata.get('year_start', 0),
                  "rating": metadata.get('rating_numeric', 0),
                  "votes": votes,
                  "director": metadata.get('director', 'N/A')
              })

          results_sorted = sorted(
              results,
              key=lambda x: x['rating'],
              reverse=True
          )[:limit]

          for result in results_sorted:
              result.pop('votes', None)

          return json.dumps(results_sorted, ensure_ascii=False, indent=2)

      except Exception as e:
          return json.dumps({"error": str(e), "message": "Error al buscar contenido mejor puntuado"}, ensure_ascii=False)


print("Tool 'get_top_rated' actualizada con filtro de votos mínimos")


Tool 'get_top_rated' actualizada con filtro de votos mínimos


## 24. Tool 4: Detalles de Contenido


In [28]:
@tool
def get_content_details(title: str) -> str:
      """
      Obtiene información detallada de una película o serie específica por título.

      Args:
          title: Título exacto o aproximado de la película/serie

      Returns:
          JSON string con los detalles completos
      """
      try:
          # Buscar por título usando búsqueda semántica
          results = vectorstore.similarity_search(
              title,
              k=3,
              namespace=NAMESPACE
          )

          if not results:
              return json.dumps({"error": f"No se encontró '{title}'"})

          # Tomar el mejor match (primero)
          doc = results[0]

          description = "N/A"
          if "Descripción: " in doc.page_content:
              description = doc.page_content.split("Descripción: ")[-1]

          details = {
              "title": doc.metadata['title'],
              "type": doc.metadata['type'],
              "genres": doc.metadata['genres'],
              "year": doc.metadata['year_start'],
              "rating": doc.metadata['rating_numeric'],
              "duration_minutes": doc.metadata['duration_minutes'],
              "director": doc.metadata.get('director', 'N/A'),
              "source": doc.metadata['source'],
              "description": description,
              "content_id": doc.metadata['content_id']
          }

          return json.dumps(details, ensure_ascii=False, indent=2)

      except Exception as e:
          return json.dumps({"error": str(e)})


print("Tool 'get_content_details' creada")


Tool 'get_content_details' creada


## Tests de Tools


In [29]:
print("=" * 60)
print("TESTS DE TOOLS")
print("=" * 60)

  # Test 1: semantic_search
print("\n1. Test de semantic_search:")
result = semantic_search.invoke({
      "query": "películas de ciencia ficción con viajes en el tiempo",
      "content_type": "Movie",
      "limit": 3
  })
print(result)

  # Test 2: filter_by_metadata
print("\n2. Test de filter_by_metadata:")
result = filter_by_metadata.invoke({
      "content_type": "Movie",
      "genre": "Action",
      "min_year": 2015,
      "min_rating": 7.5,
      "limit": 3
  })
print(result)

  # Test 3: get_top_rated
print("\n3. Test de get_top_rated:")
result = get_top_rated.invoke({
      "content_type": "Movie",
      "genre": "Drama",
      "limit": 3
  })
print(result)

  # Test 4: get_content_details
print("\n4. Test de get_content_details:")
result = get_content_details.invoke({
      "title": "Spider-Man Into the Spider-Verse"
  })
print(result)

print("\n" + "=" * 60)
print("TODOS LOS TESTS COMPLETADOS")
print("=" * 60)


TESTS DE TOOLS

1. Test de semantic_search:
[
  {
    "title": "Todo Sobre El Asado",
    "type": "Movie",
    "genres": "Documentary",
    "year": 2016.0,
    "rating": 7.0,
    "director": "Mariano Cohn, Gastón Duprat",
    "description": "This quirky examination of Argentina's culture, customs and cuisine slices into the country's traditional barbecue – which is both meal and ritual."
  },
  {
    "title": "Todo Sobre El Asado",
    "type": "Movie",
    "genres": "Documentary",
    "year": 2016.0,
    "rating": 7.0,
    "director": "Mariano Cohn, Gastón Duprat",
    "description": "This quirky examination of Argentina's culture, customs and cuisine slices into the country's traditional barbecue – which is both meal and ritual."
  },
  {
    "title": "Todo Sobre El Asado",
    "type": "Movie",
    "genres": "Documentary",
    "year": 2016.0,
    "rating": 7.0,
    "director": "Mariano Cohn, Gastón Duprat",
    "description": "This quirky examination of Argentina's culture, customs an

 ## 25. Resumen de Tools Disponibles


In [30]:
tools = [semantic_search, filter_by_metadata, get_top_rated, get_content_details]

print("=" * 60)
print("TOOLS DEL AGENTE CREADAS")
print("=" * 60)

for i, tool_func in enumerate(tools, 1):
      print(f"\n{i}. {tool_func.name}")
      print(f"   Descripción: {tool_func.description}")
      print(f"   Parámetros: {list(tool_func.args.keys())}")

print("\n" + "=" * 60)
print(f"Total: {len(tools)} herramientas listas")
print("=" * 60)


TOOLS DEL AGENTE CREADAS

1. semantic_search
   Descripción: Busca películas o series usando búsqueda semántica basada en descripción, mood o tema.

Args:
    query: Descripción de lo que el usuario busca (ej: "películas de acción emocionantes", "series románticas tristes")
    content_type: Opcional. Filtrar por tipo: "Movie" o "TV Show"
    limit: Número máximo de resultados (default: 5)

Returns:
    JSON string con los resultados encontrados
   Parámetros: ['query', 'content_type', 'limit']

2. filter_by_metadata
   Descripción: Filtra películas o series por metadata específica.

Args:
    content_type: "Movie" o "TV Show"
    genre: Género a buscar (ej: "Action", "Comedy", "Drama")
    min_year: Año mínimo de lanzamiento
    max_year: Año máximo de lanzamiento
    min_rating: Rating mínimo (0-10)
    min_duration: Duración mínima en minutos
    max_duration: Duración máxima en minutos
    limit: Número máximo de resultados

Returns:
    JSON string con los resultados filtrados
   

 ## FASE 3 COMPLETADA

  **Resumen:**
  - 4 tools implementadas con decorador @tool de LangChain
  - Todas las tools usan el vectorstore de Pinecone
  - Búsquedas semánticas + filtros de metadata funcionando
  - Tests exitosos de cada herramienta

  **Tools disponibles:**
  1. `semantic_search` - Búsqueda por descripción/mood
  2. `filter_by_metadata` - Filtros avanzados (año, rating, género, duración)
  3. `get_top_rated` - Top contenido mejor calificado
  4. `get_content_details` - Información detallada de un título


  ## 26. Imports para Agent y LLM

In [31]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import TypedDict, Annotated, Sequence
import operator


  ## 27. Configurar Llama 3.1 8B

In [32]:
llm_endpoint = HuggingFaceEndpoint(
      repo_id="meta-llama/Meta-Llama-3.1-8B-Instruct",
      huggingfacehub_api_token=HUGGINGFACE_API_TOKEN,
      temperature=0.7,
      max_new_tokens=1024,
      top_p=0.95,
      repetition_penalty=1.1
  )

llm = ChatHuggingFace(llm=llm_endpoint)

print("Probando LLM...")
test_response = llm.invoke([HumanMessage(content="Di 'Hola' en una palabra")])
print(f"Respuesta de Llama: {test_response.content}")
print("\nLLM configurado correctamente")


Probando LLM...
Respuesta de Llama: "Hola"

LLM configurado correctamente


  ## 28. Tools del LLM

In [33]:
print("Tools disponibles para el agente:")
for i, tool in enumerate(tools, 1):
      print(f"  {i}. {tool.name}")


Tools disponibles para el agente:
  1. semantic_search
  2. filter_by_metadata
  3. get_top_rated
  4. get_content_details


  ## 29. Definir State del Agente

In [34]:
class AgentState(TypedDict):
      """Estado del agente que mantiene el historial de mensajes"""
      messages: Annotated[Sequence[HumanMessage | AIMessage | SystemMessage], operator.add]

print("AgentState definido")


AgentState definido


  ## 30. System Prompt en Español


In [35]:
def chat_with_agent(user_message: str, conversation_history: list = None):
      """
      Versión simplificada que usa directamente las tools sin StateGraph.

      Args:
          user_message: Mensaje del usuario
          conversation_history: Historial previo (opcional)

      Returns:
          Respuesta del agente
      """
      if conversation_history is None:
          conversation_history = []

      system_prompt = """Eres un experto en recomendar películas y series.

  Tienes acceso a las siguientes herramientas:
  1. semantic_search(query, content_type, limit) - Busca por descripción o mood
  2. filter_by_metadata(content_type, genre, min_year, min_rating, limit) - Filtra por criterios
  3. get_top_rated(content_type, genre, limit) - Obtiene el top mejor calificado
  4. get_content_details(title) - Detalles de una película/serie específica

  Cuando el usuario pida recomendaciones:
  - Decide qué herramienta usar
  - Llama a la herramienta apropiada
  - Presenta los resultados de forma amigable en español

  Responde directamente con las recomendaciones."""

      messages = [SystemMessage(content=system_prompt)]
      messages.extend(conversation_history)
      messages.append(HumanMessage(content=user_message))

      # Analizar qué herramienta usar basado en la consulta
      query_lower = user_message.lower()

      tool_result = None

      if "top" in query_lower or "mejores" in query_lower or "mejor" in query_lower:
          content_type = "Movie" if "película" in query_lower or "pelicula" in query_lower else None
          if "serie" in query_lower:
              content_type = "TV Show"

          genre = None
          if "drama" in query_lower:
              genre = "Drama"
          elif "acción" in query_lower or "accion" in query_lower:
              genre = "Action"
          elif "comedia" in query_lower:
              genre = "Comedy"

          result = get_top_rated.invoke({
              "content_type": content_type,
              "genre": genre,
              "limit": 5
          })
          tool_result = f"Herramienta usada: get_top_rated\nResultados: {result}"

      elif any(word in query_lower for word in ["detalles", "cuéntame", "cuentame", "información", "informacion"]):
          title = user_message.split("sobre")[-1].strip() if "sobre" in query_lower else user_message

          result = get_content_details.invoke({"title": title})
          tool_result = f"Herramienta usada: get_content_details\nResultados: {result}"

      else:
          content_type = "Movie" if "película" in query_lower or "pelicula" in query_lower else None
          if "serie" in query_lower:
              content_type = "TV Show"

          result = semantic_search.invoke({
              "query": user_message,
              "content_type": content_type,
              "limit": 5
          })
          tool_result = f"Herramienta usada: semantic_search\nResultados: {result}"

      messages.append(AIMessage(content=tool_result))

      final_prompt = f"""Basándote en estos resultados de búsqueda, responde al usuario de forma amigable en español.

  Resultados: {tool_result}

  Presenta las recomendaciones de forma clara con:
  - Título
  - Año
  - Géneros
  - Rating
  - Breve descripción

  Sé conversacional y conciso."""

      messages.append(HumanMessage(content=final_prompt))

      # Obtener respuesta final del LLM
      response = llm.invoke(messages)

      return response.content


print("Función chat_with_agent creada")

Función chat_with_agent creada


  ## 33. Visualizar el Grafo

In [36]:
try:
      from IPython.display import Image, display
      display(Image(agent_executor.get_graph().draw_mermaid_png()))
      print("Grafo visualizado arriba")
except Exception as e:
      print(f"No se pudo visualizar el grafo: {e}")
      print("(Esto es opcional, el agente funciona sin visualización)")

No se pudo visualizar el grafo: name 'agent_executor' is not defined
(Esto es opcional, el agente funciona sin visualización)


  ## 35. Tests del Agente

In [37]:
print("=" * 60)
print("TEST DEL AGENTE SIMPLIFICADO")
print("=" * 60 + "\n")

print("Test 1: Mejores películas de drama")
print("-" * 60)
response1 = chat_with_agent("Recomiéndame las mejores películas de drama")
print(response1)

print("\n\n" + "=" * 60)
print("Test 2: Películas de acción recientes")
print("-" * 60)
response2 = chat_with_agent("Busco películas de acción emocionantes")
print(response2)


TEST DEL AGENTE SIMPLIFICADO

Test 1: Mejores películas de drama
------------------------------------------------------------
Lo siento, pero no encontré ninguna película de drama recomendada. ¿Quieres que busque más opciones o te recomiende otros géneros?


Test 2: Películas de acción recientes
------------------------------------------------------------
Lo siento, pero parece que la búsqueda de películas de acción emocionantes no ha dado resultado. Los resultados obtenidos son películas de drama e independientes que no coinciden con tu búsqueda.

¿Quieres intentarlo de nuevo con una búsqueda diferente? Puedes especificar más detalles, como "películas de acción con explosiones", "películas de acción con superhéroes" o "películas de acción con aventuras". Estoy aquí para ayudarte a encontrar lo que estás buscando.

Si prefieres seguir con una búsqueda diferente, dime qué tipo de películas te gustaría ver y te ayudaré a encontrar algo que se adapte a tus preferencias.


 ## FASE 4 COMPLETADA

  **Resumen:**
  - LLM: Llama 3.1 8B configurado correctamente
  - StateGraph: Construido manualmente con 2 nodos (agent, tools)
  - Memoria: InMemorySaver para mantener contexto
  - System Prompt: En español con instrucciones claras
  - Tools: 4 herramientas vinculadas al LLM
  - Tests: Agente funcionando con diferentes tipos de queries

  **Arquitectura:**
  Usuario -> HumanMessage -> Agent (LLM) -> Tool Calls -> Tools -> ToolMessages -> Agent -> Respuesta


## Celda para detectar la intencion del mensaje


In [38]:
def detectar_intencion(mensaje: str) -> str:
      """
      Detecta la intención del mensaje del usuario.

      Returns:
          'pregunta_factual_externa': Pregunta sobre eventos NO relacionados con películas/series
          'busqueda_contenido': Busca o pregunta sobre películas/series
          'conversacion_casual': Cortesías, agradecimientos, etc.
      """
      mensaje_lower = mensaje.lower()

      # PRIORIDAD 1: Conversación casual
      frases_casuales = [
          "gracias", "muchas gracias", "perfecto", "excelente", "genial",
          "ok", "vale", "bueno", "está bien", "de acuerdo",
          "cómo estás", "como estas", "qué tal", "que tal"
      ]
      if any(frase in mensaje_lower for frase in frases_casuales):
          return 'conversacion_casual'

      # PRIORIDAD 2: Menciona películas/series/documentales → SIEMPRE es búsqueda de contenido
      palabras_contenido = [
          "película", "pelicula", "film", "movie",
          "serie", "series", "tv show",
          "documental", "documentales"
      ]
      if any(palabra in mensaje_lower for palabra in palabras_contenido):
          return 'busqueda_contenido'

      # PRIORIDAD 3: Palabras de búsqueda/recomendación
      indicadores_busqueda = [
          "recomienda", "recomendacion", "recomendación",
          "busco", "quiero ver", "dame", "muestra",
          "sobre", "de", "acerca de",
          "top", "mejores", "mejor"
      ]
      if any(palabra in mensaje_lower for palabra in indicadores_busqueda):
          return 'busqueda_contenido'

      # PRIORIDAD 4: Detectar preguntas factuales
      palabras_interrogativas = [
          "quién", "quien", "qué", "que",
          "cuándo", "cuando", "dónde", "donde",
          "cómo", "como", "por qué", "porque",
          "cuál", "cual"
      ]

      es_pregunta = (mensaje.strip().startswith("¿") or mensaje.strip().startswith("?") or
                     any(palabra in mensaje_lower for palabra in palabras_interrogativas))

      if es_pregunta:
          # Verificar si menciona nombres de películas/series/actores conocidos
          # Si NO menciona contenido audiovisual → es pregunta factual externa
          palabras_cine = [
              "película", "pelicula", "serie", "film", "movie",
              "actor", "actriz", "director", "personaje",
              "temporada", "episodio", "estreno", "trama"
          ]
          if not any(palabra in mensaje_lower for palabra in palabras_cine):
              return 'pregunta_factual_externa'

      return 'busqueda_contenido'

print("Función detectar_intencion creada")


Función detectar_intencion creada


  ## 36. Versión Mejorada del Agente con Historial

In [ ]:

  import gradio as gr

  with gr.Blocks(theme=gr.themes.Soft(), title="CineBot") as demo:

      # Header
      gr.Markdown(
          """
          # 🎬 CineBot - Agente Recomendador Inteligente
          ### Descubre películas y series basadas en tus gustos
          """
      )

      with gr.Row():
          # Columna principal: Chat
          with gr.Column(scale=3):
              chatbot = gr.Chatbot(
                  label="Chat",
                  height=500,
                  show_label=False
              )

              with gr.Row():
                  msg = gr.Textbox(
                      label="Escribe tu mensaje",
                      placeholder="Ej: Dame las mejores películas de acción...",
                      scale=9,
                      show_label=False
                  )
                  send_btn = gr.Button("📤 Enviar", scale=1, variant="primary")

              clear_btn = gr.Button("🗑️ Limpiar Chat", size="sm")

          # Columna lateral: Información y ejemplos
          with gr.Column(scale=1):
              gr.Markdown("### 📊 Base de Datos")
              gr.Markdown("17,344 títulos\n\nIMDB + Netflix")

              gr.Markdown("### 💡 Ejemplos Rápidos")

              example_btns = [
                  gr.Button("🎭 Top Dramas", size="sm"),
                  gr.Button("😂 Comedias", size="sm"),
                  gr.Button("🔪 Terror", size="sm"),
                  gr.Button("❤️ Romance", size="sm"),
                  gr.Button("🚀 Sci-Fi", size="sm"),
              ]

      # Footer
      gr.Markdown(
          """
          ---
          💻 Powered by Claude AI | 🎬 Datos de IMDB & Netflix
          """
      )

      # Función de respuesta
      def respond(message, chat_history):
          bot_message = chat_with_agent_gradio(message, chat_history)
          chat_history.append({"role": "user", "content": message})
          chat_history.append({"role": "assistant", "content": bot_message})
          return "", chat_history

      # Eventos
      msg.submit(respond, [msg, chatbot], [msg, chatbot])
      send_btn.click(respond, [msg, chatbot], [msg, chatbot])
      clear_btn.click(lambda: [], None, chatbot)

      # Botones de ejemplo
      example_btns[0].click(lambda: "Dame el top 5 de series de drama", None, msg)
      example_btns[1].click(lambda: "Películas de comedia divertidas", None, msg)
      example_btns[2].click(lambda: "Las mejores películas de terror", None, msg)
      example_btns[3].click(lambda: "Series románticas", None, msg)
      example_btns[4].click(lambda: "Películas de ciencia ficción", None, msg)

  print("Interfaz Gradio avanzada creada")

Función chat_with_agent_gradio actualizada con límites dinámicos


  ## 37. Interfaz Gradio

In [54]:
import gradio as gr
                                                                                                                                                                                                # Tema Netflix personalizado
netflix_theme = gr.themes.Soft(
      primary_hue="red",
      secondary_hue="slate",
      neutral_hue="slate",
  ).set(
      body_background_fill="#141414",  # Negro Netflix
      body_text_color="#ffffff",
      block_background_fill="#221f1f",  # Gris oscuro Netflix
      block_title_text_color="#ffffff",
      block_label_text_color="#ffffff",
      input_background_fill="#333333",
      button_primary_background_fill="#E50914",  # Rojo Netflix
      button_primary_background_fill_hover="#F40612",  # Rojo más brillante al hover
      button_primary_text_color="#ffffff",
  )

with gr.Blocks(theme=netflix_theme, title="CineBot") as demo:

      # Header
      gr.Markdown(
          """
          #  Agente Recomendador de Películas y Series
          ### Descubre películas y series basadas en tus gustos
          """
      )

      with gr.Row():
          with gr.Column(scale=3):
              chatbot = gr.Chatbot(
                  label="Chat",
                  height=500,
                  show_label=False
              )

              with gr.Row():
                  msg = gr.Textbox(
                      label="Escribe tu mensaje",
                      placeholder="Ej: Dame las mejores películas de acción...",
                      scale=9,
                      show_label=False
                  )
                  send_btn = gr.Button("Enviar", scale=1, variant="primary")

              clear_btn = gr.Button("Limpiar Chat", size="sm")

          # Columna lateral: Información y ejemplos
          with gr.Column(scale=1):
              gr.Markdown("###  Base de Datos")
              gr.Markdown("**17,344 títulos**\n\nIMDB + Netflix")

              gr.Markdown("###  Ejemplos Rápidos")

              example_btns = [
                  gr.Button(" Top Dramas", size="sm"),
                  gr.Button(" Comedias", size="sm"),
                  gr.Button(" Terror", size="sm"),
                  gr.Button(" Romance", size="sm"),
                  gr.Button(" Sci-Fi", size="sm"),
              ]

      # Footer
      gr.Markdown(
          """
          ---
          Datos de IMDB & Netflix 
          """
      )

      # Función de respuesta
      def respond(message, chat_history):
          bot_message = chat_with_agent_gradio(message, chat_history)
          chat_history.append({"role": "user", "content": message})
          chat_history.append({"role": "assistant", "content": bot_message})
          return "", chat_history

      # Eventos
      msg.submit(respond, [msg, chatbot], [msg, chatbot])
      send_btn.click(respond, [msg, chatbot], [msg, chatbot])
      clear_btn.click(lambda: [], None, chatbot)

      # Botones de ejemplo
      example_btns[0].click(lambda: "Dame el top 5 de series de drama", None, msg)
      example_btns[1].click(lambda: "Películas de comedia divertidas", None, msg)
      example_btns[2].click(lambda: "Las mejores películas de terror", None, msg)
      example_btns[3].click(lambda: "Series románticas", None, msg)
      example_btns[4].click(lambda: "Películas de ciencia ficción", None, msg)

print("Interfaz Gradio con tema Netflix creada")


Interfaz Gradio con tema Netflix creada


## 38. Lanzar la Aplicación


In [55]:
demo.launch(
      share=False,
      server_name="127.0.0.1",
      server_port=7895,
      show_error=True
  )


* Running on local URL:  http://127.0.0.1:7895
* To create a public link, set `share=True` in `launch()`.
